In [5]:
%%file ../libs/estimateGaussian.m
function [mu, sigma2] = estimateGaussian(X)
    [m, n] = size(X);

    mu = zeros(n, 1);
    sigma2 = zeros(n, 1);

    mu = (sum(X) ./ m)';
    sigma2 = (sum((X - mu') .^ 2) ./ m)';
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/estimateGaussian.m'.


In [37]:
%%file ../libs/estimateGaussianDensity.m
function p = estimateGaussianDensity(X, mu, sigma2)
    n = length(mu);
    X = X - mu';
    p = prod((2*pi)^(-0.5)*(sigma2').^(-0.5) .* exp(-0.5*(X .* X)./sigma2'), 2);
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/estimateGaussianDensity.m'.


In [38]:
%%file ../libs/multivariateGaussian.m
function p = multivariateGaussian(X, mu, sigma2)
    n = length(mu);
    if(size(sigma2, 2) == 1 || size(sigma, 1) == 1)
        Sigma2 = diag(sigma2);
    end

    X = X - mu';
    p = (2 * pi)^(-n/2) * det(Sigma2)^(-0.5) * exp(-0.5*sum(X*pinv(Sigma2) .* X, 2));
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/multivariateGaussian.m'.


In [3]:
%%file ../libs/visualizeFit.m
function visualizeFit(X, mu, sigma2, density_fn)

    [X1, X2] = meshgrid(0:.5:35);

    Z = density_fn([X1(:) X2(:)], mu, sigma2);
    Z = reshape(Z, size(X1));

    plot(X(:, 1), X(:, 2), 'bx');
    hold on;
    if (sum(isinf(Z)) == 0)
        contour(X1, X2, Z, 10 .^(-20:3:0)');
    end
    hold off;
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/visualizeFit.m'.


In [23]:
%%file ../libs/selectThreshold.m
function [bestEpsilon bestF1] = selectThreshold(yval, pval)

    bestEpsilon = 0;
    bestF1 = 0;
    F1 = 0;

    stepsize = (max(pval) - min(pval)) / 1000;
    for epsilon = min(pval):stepsize:max(pval)
        pred = pval < epsilon;
        tp = sum((pred == 1) & (yval == 1));
        fp = sum((pred == 1) & (yval == 0));
        fn = sum((pred == 0) & (yval == 1));
        prec = tp / (tp + fp);
        rec = tp / (tp + fn);
        F1 = (2 * prec * rec) / (prec + rec);

        if F1 > bestF1
            bestF1 = F1;
            bestEpsilon = epsilon;
        end
    end
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/selectThreshold.m'.


In [24]:
%%file ../libs/plotFeatureHist.m
function plotFeatureHist(X, ncols=3, nbins=25)
    nfeatures = size(X, 2);
    ncols = min(nfeatures, ncols);
    nrows = ceil(nfeatures / ncols);
    for i = 1:nfeatures
        subplot(nrows, ncols, i);
        hist(X(:, i), nbins);
        title(sprintf('feature x%d', i));
    end
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/plotFeatureHist.m'.


In [23]:
%%file ../libs/cofiCostFn.m
function J = cofiCostFn(X, Theta, Y, R, lambda)
    % X = m x n
    % Theta = u x n
    % Y = m x u
    % R = m x u
    J = 0.5 * sum(sum(((X * Theta' - Y) .^ 2) .* R));

    % reg
    reg = 0.5 * lambda * (sum(sum(Theta .^2)) + sum(sum(X .^2)));

    J += reg;
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/cofiCostFn.m'.


In [25]:
%%file ../libs/cofiGradFn.m
function grad = cofiGradFn(X, Theta, Y, R, lambda)
    % X = m x n
    % Theta = u x n
    % Y = m x u
    % R = m x u

    X_grad = zeros(size(X));
    Theta_grad = zeros(size(Theta));

    common = (X * Theta' - Y) .* R;

    X_grad = common * Theta + lambda * X;
    Theta_grad = common' * X + lambda * Theta;

    grad = [X_grad(:); Theta_grad(:)];
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/cofiGradFn.m'.


In [26]:
%%file ../libs/cofiCostnGradFn.m
function [J, grad] = cofiCostnGradFn(params, Y, R, num_users, num_movies, num_features, lambda)
    X = reshape(params(1:num_movies*num_features), num_movies, num_features);
    Theta = reshape(params(num_movies*num_features+1:end), num_users, num_features);

    J = 0;
    X_grad = zeros(size(X));
    Theta_grad = zeros(size(Theta));

    J = cofiCostFn(X, Theta, Y, R, lambda);
    grad = cofiGradFn(X, Theta, Y, R, lambda);
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/cofiCostnGradFn.m'.


In [32]:
%%file ../libs/normalizeRatings.m
function [Ynorm, Ymean] = normalizeRatings(Y, R)
    [m, n] = size(Y);
    Ymean = zeros(m, 1);
    Ynorm = zeros(size(Y));
    for i = 1:m
        idx = find(R(i, :) == 1);
        Ymean(i) = mean(Y(i, idx));
        Ynorm(i, idx) = Y(i, idx) - Ymean(i);
    end
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/normalizeRatings.m'.


In [37]:
%%file ../libs/topkRelatedMovie.m
function ix = topkRelatedMovies(X, mi, k)
    m = size(X, 1);
    xi = X(mi, :);

    error = sum((X - xi) .^2, 2);
    [r, ix] = sort(error);
    ix = ix(2:k+1);
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/topkRelatedMovie.m'.


# Helper

In [31]:
%%file ../libs/loadMovieList.m
function movieList = loadMovieList()
    fd = fopen('../../data/movie_ids.txt');
    n = 1682;
    movieList = cell(n, 1);
    for i = 1:n
        line = fgets(fd);
        [idx, movieName] = strtok(line, ' ');
        movieList{i} = strtrim(movieName);
    end
    fclose(fd);
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/loadMovieList.m'.


In [28]:
%%file ../libs/computeNumericalGradient.m
function numgrad = computeNumericalGradient(J, theta)
    numgrad = zeros(size(theta));
    perturb = zeros(size(theta));
    e = 1e-4;

    for p = 1:numel(theta)
        perturb(p) = e;
        loss1 = J(theta - perturb);
        loss2 = J(theta + perturb);

        numgrad(p) = (loss2 - loss1) / (2*e);
        perturb(p) = 0;
    end
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/computeNumericalGradient.m'.


In [30]:
%%file ../libs/checkCostFunction.m
function checkCostFunction(lambda=0)

    X_t = rand(4, 3);
    Theta_t = rand(5, 3);

    Y = X_t * Theta_t';
    Y(rand(size(Y)) > 0.5) = 0;
    R = zeros(size(Y));
    R(Y ~= 0) = 1;

    % run gradient checking
    X = randn(size(X_t));
    Theta = randn(size(Theta_t));
    num_users = size(Y, 2);
    num_movies = size(Y, 1);
    num_features = size(Theta_t, 2);

    numgrad = computeNumericalGradient(...
                @(t) cofiCostnGradFn(t, Y, R, num_users, num_movies, num_features, lambda), ...
                [X(:); Theta(:)]);
    [cost, grad] = cofiCostnGradFn([X(:); Theta(:)], Y, R, num_users, num_movies, num_features, lambda);
    disp([numgrad grad]);
    fprintf(['The above two columns you get should be very similar.\n' ...
            '(Left-Your Numerical Gradient, Right-Analytical Gradient)\n\n']);

    diff = norm(numgrad-grad)/norm(numgrad+grad);
    fprintf(['If your backpropagation implementation is correct, then \n' ...
            'the relative difference will be small (less than 1e-9). \n' ...
            '\nRelative Difference: %g\n'], diff);

end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/checkCostFunction.m'.


In [33]:
%%file ../libs/fmincg.m
function [X, fX, i] = fmincg(f, X, options, P1, P2, P3, P4, P5)
    % Minimize a continuous differentialble multivariate function. Starting point
    % is given by "X" (D by 1), and the function named in the string "f", must
    % return a function value and a vector of partial derivatives. The Polack-
    % Ribiere flavour of conjugate gradients is used to compute search directions,
    % and a line search using quadratic and cubic polynomial approximations and the
    % Wolfe-Powell stopping criteria is used together with the slope ratio method
    % for guessing initial step sizes. Additionally a bunch of checks are made to
    % make sure that exploration is taking place and that extrapolation will not
    % be unboundedly large. The "length" gives the length of the run: if it is
    % positive, it gives the maximum number of line searches, if negative its
    % absolute gives the maximum allowed number of function evaluations. You can
    % (optionally) give "length" a second component, which will indicate the
    % reduction in function value to be expected in the first line-search (defaults
    % to 1.0). The function returns when either its length is up, or if no further
    % progress can be made (ie, we are at a minimum, or so close that due to
    % numerical problems, we cannot get any closer). If the function terminates
    % within a few iterations, it could be an indication that the function value
    % and derivatives are not consistent (ie, there may be a bug in the
    % implementation of your "f" function). The function returns the found
    % solution "X", a vector of function values "fX" indicating the progress made
    % and "i" the number of iterations (line searches or function evaluations,
    % depending on the sign of "length") used.
    %
    % Usage: [X, fX, i] = fmincg(f, X, options, P1, P2, P3, P4, P5)
    %
    % See also: checkgrad 
    %
    % Copyright (C) 2001 and 2002 by Carl Edward Rasmussen. Date 2002-02-13
    %
    %
    % (C) Copyright 1999, 2000 & 2001, Carl Edward Rasmussen
    % 
    % Permission is granted for anyone to copy, use, or modify these
    % programs and accompanying documents for purposes of research or
    % education, provided this copyright notice is retained, and note is
    % made of any changes that have been made.
    % 
    % These programs and documents are distributed without any warranty,
    % express or implied.  As the programs were written for research
    % purposes only, they have not been tested to the degree that would be
    % advisable in any important application.  All use of these programs is
    % entirely at the user's own risk.
    %
    % [ml-class] Changes Made:
    % 1) Function name and argument specifications
    % 2) Output display
    %

    % Read options
    if exist('options', 'var') && ~isempty(options) && isfield(options, 'MaxIter')
        length = options.MaxIter;
    else
        length = 100;
    end


    RHO = 0.01;                            % a bunch of constants for line searches
    SIG = 0.5;       % RHO and SIG are the constants in the Wolfe-Powell conditions
    INT = 0.1;    % don't reevaluate within 0.1 of the limit of the current bracket
    EXT = 3.0;                    % extrapolate maximum 3 times the current bracket
    MAX = 20;                         % max 20 function evaluations per line search
    RATIO = 100;                                      % maximum allowed slope ratio

    argstr = ['feval(f, X'];                      % compose string used to call function
    for i = 1:(nargin - 3)
        argstr = [argstr, ',P', int2str(i)];
    end
    argstr = [argstr, ')'];

    if max(size(length)) == 2, red=length(2); length=length(1); else red=1; end
    S=['Iteration '];

    i = 0;                                            % zero the run length counter
    ls_failed = 0;                             % no previous line search has failed
    fX = [];
    [f1 df1] = eval(argstr);                      % get function value and gradient
    i = i + (length<0);                                            % count epochs?!
    s = -df1;                                        % search direction is steepest
    d1 = -s'*s;                                                 % this is the slope
    z1 = red/(1-d1);                                  % initial step is red/(|s|+1)

    while i < abs(length)                                      % while not finished
        i = i + (length>0);                                      % count iterations?!

        X0 = X; f0 = f1; df0 = df1;                   % make a copy of current values
        X = X + z1*s;                                             % begin line search
        [f2 df2] = eval(argstr);
        i = i + (length<0);                                          % count epochs?!
        d2 = df2'*s;
        f3 = f1; d3 = d1; z3 = -z1;             % initialize point 3 equal to point 1
        if length>0, M = MAX; else M = min(MAX, -length-i); end
        success = 0; limit = -1;                     % initialize quanteties
        while 1
            while ((f2 > f1+z1*RHO*d1) || (d2 > -SIG*d1)) && (M > 0) 
                limit = z1;                                         % tighten the bracket
                if f2 > f1
                  z2 = z3 - (0.5*d3*z3*z3)/(d3*z3+f2-f3);                 % quadratic fit
                else
                  A = 6*(f2-f3)/z3+3*(d2+d3);                                 % cubic fit
                  B = 3*(f3-f2)-z3*(d3+2*d2);
                  z2 = (sqrt(B*B-A*d2*z3*z3)-B)/A;       % numerical error possible - ok!
                end
                if isnan(z2) || isinf(z2)
                  z2 = z3/2;                  % if we had a numerical problem then bisect
                end
                z2 = max(min(z2, INT*z3),(1-INT)*z3);  % don't accept too close to limits
                z1 = z1 + z2;                                           % update the step
                X = X + z2*s;
                [f2 df2] = eval(argstr);
                M = M - 1; i = i + (length<0);                           % count epochs?!
                d2 = df2'*s;
                z3 = z3-z2;                    % z3 is now relative to the location of z2
            end
            if f2 > f1+z1*RHO*d1 || d2 > -SIG*d1
                break;                                                % this is a failure
            elseif d2 > SIG*d1
                success = 1; break;                                             % success
            elseif M == 0
                break;                                                          % failure
            end
            A = 6*(f2-f3)/z3+3*(d2+d3);                      % make cubic extrapolation
            B = 3*(f3-f2)-z3*(d3+2*d2);
            z2 = -d2*z3*z3/(B+sqrt(B*B-A*d2*z3*z3));        % num. error possible - ok!
            if ~isreal(z2) || isnan(z2) || isinf(z2) || z2 < 0   % num prob or wrong sign?
                if limit < -0.5                               % if we have no upper limit
                    z2 = z1 * (EXT-1);                 % the extrapolate the maximum amount
                else
                    z2 = (limit-z1)/2;                                   % otherwise bisect
                end
            elseif (limit > -0.5) && (z2+z1 > limit)          % extraplation beyond max?
                z2 = (limit-z1)/2;                                               % bisect
            elseif (limit < -0.5) && (z2+z1 > z1*EXT)       % extrapolation beyond limit
                z2 = z1*(EXT-1.0);                           % set to extrapolation limit
            elseif z2 < -z3*INT
                z2 = -z3*INT;
            elseif (limit > -0.5) && (z2 < (limit-z1)*(1.0-INT))   % too close to limit?
                z2 = (limit-z1)*(1.0-INT);
            end
            f3 = f2; d3 = d2; z3 = -z2;                  % set point 3 equal to point 2
            z1 = z1 + z2; X = X + z2*s;                      % update current estimates
            [f2 df2] = eval(argstr);
            M = M - 1; i = i + (length<0);                             % count epochs?!
            d2 = df2'*s;
        end                                                      % end of line search

        if success                                         % if line search succeeded
            f1 = f2; fX = [fX' f1]';
            fprintf('%s %4i | Cost: %4.6e\r', S, i, f1);
            s = (df2'*df2-df1'*df2)/(df1'*df1)*s - df2;      % Polack-Ribiere direction
            tmp = df1; df1 = df2; df2 = tmp;                         % swap derivatives
            d2 = df1'*s;
            if d2 > 0                                      % new slope must be negative
                s = -df1;                              % otherwise use steepest direction
                d2 = -s'*s;    
            end
            z1 = z1 * min(RATIO, d1/(d2-realmin));          % slope ratio but max RATIO
            d1 = d2;
            ls_failed = 0;                              % this line search did not fail
        else
          X = X0; f1 = f0; df1 = df0;  % restore point from before failed line search
          if ls_failed || i > abs(length)          % line search failed twice in a row
              break;                             % or we ran out of time, so we give up
          end
          tmp = df1; df1 = df2; df2 = tmp;                         % swap derivatives
          s = -df1;                                                    % try steepest
          d1 = -s'*s;
          z1 = 1/(1-d1);                     
          ls_failed = 1;                                    % this line search failed
        end
        if exist('OCTAVE_VERSION')
            fflush(stdout);
        end
      end
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex8/octave/libs/fmincg.m'.
